```{contents}
```
## **Local Node State in LangGraph**

In LangGraph, **Local Node State** refers to **temporary, node-scoped data** that exists only during the execution of a single node and is **not persisted** into the global shared graph state unless explicitly returned.
It enables **clean computation, isolation, performance optimization, and safe intermediate reasoning**.

---

### **1. Motivation and Intuition**

LangGraph maintains a **global state** shared across all nodes.
However, complex nodes often need **private working memory**:

| Need                   | Why                          |
| ---------------------- | ---------------------------- |
| Intermediate variables | Avoid polluting global state |
| Scratch computations   | Safer experimentation        |
| Temporary caches       | Performance                  |
| Sensitive data         | Prevent leakage              |

Local node state provides this **private workspace**.

---

### **2. Conceptual Model**

```
Global Graph State  ─────────────┐
                                 ↓
                           [   Node   ]
                          ( Local State )
                                 ↓
Global Graph State  <─────── returned updates
```

Only the returned values affect the global state.

---

### **3. How Local Node State Works**

A node function receives:

```python
def node_fn(global_state):
    # local variables = local node state
```

All Python variables inside the function are **local node state** unless returned.

```python
def analyze(state):
    temp_score = expensive_analysis(state["input"])  # local
    intermediate = normalize(temp_score)              # local
    
    return {"result": intermediate}   # only this enters global state
```

`temp_score` and `intermediate` never leave the node.

---

### **4. Why This Matters**

| Property    | Benefit                      |
| ----------- | ---------------------------- |
| Isolation   | Prevents unintended coupling |
| Security    | Sensitive data not persisted |
| Efficiency  | Smaller global state         |
| Correctness | Avoids state corruption      |
| Debugging   | Clear data flow              |

---

### **5. Local vs Global State**

| Feature     | Local Node State      | Global State    |
| ----------- | --------------------- | --------------- |
| Scope       | Single node execution | Entire graph    |
| Lifetime    | One execution step    | Whole run       |
| Persistence | Not persisted         | Checkpointed    |
| Visibility  | Node only             | All nodes       |
| Safety      | High                  | Must be managed |

---

### **6. Example: Clean Multi-Step Node**

```python
def planner(state):
    prompt = build_prompt(state["goal"])      # local
    raw_plan = llm(prompt)                    # local
    structured = parse_plan(raw_plan)         # local
    
    return {"plan": structured}               # only plan persists
```

---

### **7. Performance Optimization with Local State**

Expensive objects should remain local:

```python
def tool_node(state):
    model = load_large_model()   # local, not persisted
    output = model.run(state["data"])
    return {"tool_result": output}
```

This avoids bloating checkpoints.

---

### **8. Local State in Cycles & Agents**

In loops and multi-agent systems:

* Local state handles **thinking**
* Global state handles **memory**

This separation prevents **memory explosion** and enables long-running agents.

---

### **9. Anti-Patterns to Avoid**

| Anti-Pattern              | Why Bad           |
| ------------------------- | ----------------- |
| Returning everything      | Bloats state      |
| Storing raw LLM outputs   | Noisy memory      |
| Persisting temporary data | Slows checkpoints |
| Sharing sensitive data    | Security risk     |

---

### **10. Mental Model**

> **Local Node State = CPU registers**
> **Global State = RAM**

Only what must persist should be written to global memory.

---

### **11. When to Use Local Node State**

Use local state for:

* Intermediate reasoning
* Parsing & normalization
* Temporary caches
* Tool setup objects
* Scratchpad thinking

Use global state for:

* Decisions
* Plans
* Results
* Long-term memory

---

### **12. Key Design Principle**

> **Persist only what future nodes must know.**

This single rule yields **fast, safe, scalable LangGraph systems**.


### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    user_query: str
    decision: str

def analyze(state: State):
    # -------- Local Node State --------
    tokens = state["user_query"].split()         # local
    complexity = len(tokens)                     # local
    importance_score = complexity * 1.7          # local
    reasoning_trace = f"score={importance_score}"# local
    # ---------------------------------

    if importance_score > 5:
        decision = "complex"
    else:
        decision = "simple"

    return {"decision": decision}


In [2]:
builder = StateGraph(State)

builder.add_node("analyze", analyze)
builder.set_entry_point("analyze")
builder.add_edge("analyze", END)

graph = builder.compile()


In [3]:
result = graph.invoke({"user_query": "Explain local node state in langgraph"})
print(result)


{'user_query': 'Explain local node state in langgraph', 'decision': 'complex'}
